In [7]:
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import pandas as pd

class GAOptimize(object):
    def __init__(self, no_cities, total_count, iteration, data):
        self.no_cities = no_cities
        self.total_count = total_count
        self.scores = []
        self.iteration = iteration
        self.locale = data
        self.choose_probability = 0.89
        self.mutate_probability = 0.03
        # compute distance matirx
        self.dis_mat = self.comp_dis_matrix(no_cities, data)
        self.greed_init = self.greedy_initialize(self.dis_mat,total_count,no_cities)
        # best route after initialization
        scores = self.comp_adp(self.greed_init)
        sort_index = np.argsort(-scores)
        init_best = self.greed_init[sort_index[0]]
        init_best = self.locale[init_best]

        # Store the results of each iteration and draw a convergence graph
        self.loop_x = [0]
        self.loop_y = [1. / scores[sort_index[0]]]
        self.mean_length = None
        self.median_length = None
    def random_initialize(self, total_count, no_cities):
        tsp = [i for i in range(no_cities)]
        result = []
        for j in range(total_count):
            random.shuffle(tsp)
            result.append(tsp.copy())
        return result

    def greedy_initialize(self, dis_mat, total_count, no_cities):
        start_idx = 0
        result = []
        for i in range(total_count):
            rest = [i for i in range(0, no_cities)]
            # Generate starting points 
            if start_idx >= no_cities:
                start_idx = np.random.randint(0, no_cities)
                result.append(result[start_idx].copy())
                continue
            curr = start_idx
            rest.remove(curr)
            # Find nearest neighbour route
            result_one = [curr]
            while len(rest) != 0:
                min_dist = math.inf
                dist_select = -1
                for k in rest:
                    if dis_mat[curr][k] < min_dist:
                        min_dist = dis_mat[curr][k]
                        dist_select = k

                curr = dist_select
                result_one.append(dist_select)
                rest.remove(dist_select)
            result.append(result_one)
            start_idx += 1
        return result
    #Calculate distance between cities
    def comp_dis_matrix(self, no_cities, locale):
        dis_mat = np.zeros((no_cities, no_cities))

        for i in range(no_cities):
            for j in range(no_cities):
                if i == j:
                    dis_mat[i][j] = np.inf
                    continue
                c = locale[i]
                d = locale[j]
                tsp = np.sqrt(sum([(x[0] - x[1]) ** 2 for x in zip(c, d)]))
                dis_mat[i][j] = tsp
        return dis_mat

    # Calculate route length
    def comp_routelength(self, route, dis_mat):
        try:
            c = route[0]
            d = route[-1]

        except:
            import pdb
            pdb.set_trace()
        result = dis_mat[c][d]
        for i in range(len(route) - 1):
            c = route[i]
            d = route[i + 1]
            result += dis_mat[c][d]
        return result

    # Calculate population fitness
    def comp_adp(self, tour):
        adp = []
        for i in tour:
            if isinstance(i, int):
                import pdb
                pdb.set_trace()
            length = self.comp_routelength(i, self.dis_mat)
            adp.append(1.0 / length)
        self.mean_length = 1.0/np.mean(adp)
        self.median_length = 1.0/np.median(adp)
        return np.array(adp)

    def swap(self, list1, list2):
        index = len(list1)
        list = list1 + list2
        list = list[::-1]
        return list[:index], list[index:]

    def cross(self, x, y):
        len_ = len(x)
        assert len(x) == len(y)
        route_list = [t for t in range(len_)]
        order = list(random.sample(route_list, 2))
        order.sort()
        start, end = order

        # Find the conflict points and save their subscripts 
        tsp = x[start:end]
        x_conflict_index = []
        for sub in tsp:
            index = y.index(sub)
            if not (index >= start and index < end):
                x_conflict_index.append(index)

        y_confict_index = []
        tsp = y[start:end]
        for sub in tsp:
            index = x.index(sub)
            if not (index >= start and index < end):
                y_confict_index.append(index)

        assert len(x_conflict_index) == len(y_confict_index)

        
        tsp = x[start:end].copy()
        x[start:end] = y[start:end]
        y[start:end] = tsp

        # resolve conflict
        for k in range(len(x_conflict_index)):
            i = x_conflict_index[k]
            j = y_confict_index[k]
            y[i], x[j] = x[j], y[i]

        assert len(set(x)) == len_ and len(set(y)) == len_
        return list(x), list(y)

    def parent(self, scores, gene_choose_probability):
        sort_index = np.argsort(-scores).copy()
        sort_index = sort_index[0:int(gene_choose_probability * len(sort_index))]
        parents = []
        parents_score = []
        for i in sort_index:
            parents.append(self.greed_init[i])
            parents_score.append(scores[i])

        return parents, parents_score

    def gene_choose(self, score, pop_select):
        sum_score = sum(score)
        score_ratio = [i * 1.0 / sum_score for i in score]
        rand1 = np.random.rand()
        rand2 = np.random.rand()
        for j, k in enumerate(score_ratio):
            if rand1 >= 0:
                rand1 -= k
                if rand1 < 0:
                    index1 = j
            if rand2 >= 0:
                rand2 -= k
                if rand2 < 0:
                    index2 = j
            if rand1 < 0 and rand2 < 0:
                break
        return list(pop_select[index1]), list(pop_select[index2])

    def mutate(self, gene):
        route_list = [i for i in range(len(gene))]
        order = list(random.sample(route_list, 2))
        start, end = min(order), max(order)
        tsp = gene[start:end]
        
        tsp = tsp[::-1]
        gene[start:end] = tsp
        return list(gene)

    def ga(self):
        # get high quality parent

        scores = self.comp_adp(self.greed_init)
        # Select some excellent individuals as the parent candidate set
        parents, parents_score = self.parent(scores, self.choose_probability)
        tsp_best_one = parents[0]
        tsp_best_score = parents_score[0]
        # new population
        fit = parents.copy()

        # Generate a new population
        while len(fit) < self.total_count:
            # Roulette method
            gene_x, gene_y = self.gene_choose(parents_score, parents)
            # cross over
            gene_x_new, gene_y_new = self.cross(gene_x, gene_y)
            # Mutation
            if np.random.rand() < self.mutate_probability:
                gene_x_new = self.mutate(gene_x_new)
            if np.random.rand() < self.mutate_probability:
                gene_y_new = self.mutate(gene_y_new)
            x_adp = 1. / self.comp_routelength(gene_x_new, self.dis_mat)
            y_adp = 1. / self.comp_routelength(gene_y_new, self.dis_mat)
            # Put those with high fitness into the population
            if x_adp > y_adp and (not gene_x_new in fit):
                fit.append(gene_x_new)
            elif x_adp <= y_adp and (not gene_y_new in fit):
                fit.append(gene_y_new)

        self.greed_init = fit

        return tsp_best_one, tsp_best_score

    def run(self):
        results_dict = {
            "generation": [],
            "best_fitness": [],
            "mean_fitness": [],
            "median_fitness": [],

        }
        BEST_LIST = None
        best_score = -math.inf
        self.best_record = []
        for j in range(1, self.iteration + 1):
            random.Random(j)
            tsp_best_one, tsp_best_score = self.ga()
            self.loop_x.append(j)
            self.loop_y.append(1. / tsp_best_score)
            if tsp_best_score > best_score:
                best_score = tsp_best_score
                BEST_LIST = tsp_best_one
            self.best_record.append(1./best_score)
            #print("here",j,best_score,self.mean_length,self.median_length,1./best_score)
            results_dict["generation"].append(j)
            results_dict["best_fitness"].append(1/best_score)
            results_dict["mean_fitness"].append(self.mean_length)
            results_dict["median_fitness"].append(self.median_length)

        print(1./best_score)
        return self.locale[BEST_LIST], 1. / best_score,results_dict


# Read the TSP data file
def read_tsp(route):
    rows = open(route, 'r').readlines()
    assert 'NODE_COORD_SECTION\n' in rows
    idx = rows.index('NODE_COORD_SECTION\n')
    dist = rows[idx + 1:-1]
    tsp = []
    for i in dist:
        i = i.strip().split(' ')
        if i[0] == 'EOF':
            continue
        tspline = []
        for k in i:
            if k == '':
                continue
            else:
                tspline.append(float(k))
        if tspline == []:
            continue
        tsp.append(tspline)
    dist = tsp
    return dist


dataset = read_tsp('/content/st70.tsp')

dataset = np.array(dataset)
dataset = dataset[:, 1:]
Best, Best_route = math.inf, None
all_results = []
import pandas as pd
for i in range(0,30):
 random.seed(i)
 model = GAOptimize(no_cities=dataset.shape[0], total_count=500, iteration=50, data=dataset.copy())
 route, route_len,results = model.run()
 #print(pd.DataFrame(results))
 all_results.append(pd.DataFrame(results))

final_results = pd.concat(all_results, axis=1)

if route_len < Best:
    Best = route_len
    Best_route = route
Best_route = np.vstack([Best_route, Best_route[0]])
plt.plot(Best_route[:, 0], Best_route[:, 1], marker='o', markerfacecolor='red')
plt.title('st70:GA result')
plt.show()


752.3771682902174


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
final_results.to_csv('file_ga_50iter_new_500pop.csv', index=False)

In [ ]:
final= final_results.drop('generation', axis=1).reset_index()


In [ ]:
final

In [ ]:
import numpy as np

def parseData(data, firstcolumn, noRuns):
    col = firstcolumn
    
    allstats = (data.shape[1]-1)/noRuns   # how many stats were collected. Omit the first column (Generations)
    cols = np.arange(col, noRuns*allstats+1, allstats, dtype=int)
    #print(cols)
    subdata = data.iloc[:,cols]
    # print(subdata)
    noGens = data.shape[0]
    pdata = np.zeros((noGens, 4))
    for i in range(noGens):
        pdata[i,0] = i+1
        pdata[i,1] = np.mean(subdata.iloc[i,:].mean()) # mean(subdata[i,])
        pdata[i,2] = 1.96*np.std(subdata.iloc[i,:])/np.sqrt(noRuns) # 1.96*sd(rowMeans(subdata))/sqrt(noRuns) # compute the length of error bar. 
        pdata[i,3] = np.min(subdata.iloc[i,:]) # mean(subdata[i,])
    return pdata


In [ ]:
data = parseData(final, 1, 30)
mean_second_col = np.mean(data[:,1])
min_second_col = np.min(data[:,1])

print("mean ",mean_second_col)
print("min ",min_second_col)

mean  760.5954273671217
min  759.9774807723265
